In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy as sql
import pymysql
import sklearn.linear_model as skllm
import statsmodels.api as sm
import bin_time_data as bd
import query_database as qdb
%matplotlib inline

In [2]:
sns.set_context('poster')

In [3]:
#connect to db
zidisha_db = qdb.QueryDatabase('zidisha')

In [12]:
sql_dir = 'SQL_queries'

#new accepted borrower account creation dates
sql_query_file = 'borrowers_w_countries.sql'
df_new_borrowers = zidisha_db.source_query(sql_dir, sql_query_file)
df_new_borrowers.insert(0, 'number', range(len(df_new_borrowers)))
#df_new_borrowers = pd.DataFrame({'number': range(len(df_new_borrowers)), 'country': df_new_borrowers.country_name.values}, index=df_new_borrowers.created_at.values)

#borrower first loan creation dates
sql_query_file = 'first_loans_w_countries.sql'
df_first_loans = zidisha_db.source_query(sql_dir, sql_query_file)



In [ ]:
df_new_borrowers.columns.tolist()[0]

In [34]:
df_first_loans.pivot(index='first_loan_date',columns='country_name')

KeyError: 'first_loan_date'

In [38]:
df_first_loans['index']

KeyError: 'index'

In [28]:
tlag = pd.DataFrame({'tdiff': df_first_loans.index.values - df_first_loans.created_at.values}, index=df_first_loans.index)
tlag = pd.DataFrame({'tdiff': tlag.tdiff.values}, index=tlag.index.values)
tlag.plot()

TypeError: Empty 'DataFrame': no numeric data to plot

In [24]:
x=np.timedelta64(tlag.tdiff[0])
date=x.astype('timedelta64[s]')
date.astype(int)

781